# Minhash for recommendations

In the [previous notebook](04-minhash.ipynb) we saw how Minhash can be used to approximate the similarity of sets. In this notebook we will see that Minhash can also be used to make recommendations. 

We illustrate this technique using a data set which contains users' listening history from a music streaming service. If you're interested in how we generated this data, take a look at [this notebook](99a-data-generator.ipynb). 

In [ ]:
import os
import pandas as pd

path = 'data/' 
files = os.listdir(path)
files_data = [i for i in files if i.startswith('userdat')][0:1] #listing all files in the directory of the correct form
df = pd.DataFrame(columns=['user', 'artist','plays'])
for j in files_data:
    pseudo_data = pd.read_parquet('data/'+j)
    df = pd.concat([df, pseudo_data])
    print(df.shape)

The data contains three columns and over three million rows. Let's take a closer look at a sample of the data. 

In [ ]:
df.sample(10, random_state=1)

The first column is an integer representing a user id, the second is an integer representing an artist name, and the third column is an integer indicating how many times the user listened to the artist. 

We take one pass through the data to identify all unique artists all unique users and artists. 

In [ ]:
df['artist'] = df['artist'].astype('int')
df['user'] = df['user'].astype('int')

In [ ]:
artists = df['artist'].unique()
users = df['user'].unique()

In [ ]:
print("There are ", len(artists), " artists and ", len(users), " users in our data." , sep="")

We map those user names to unique integers and store those in a dictionary.

In [ ]:
artists.dtype

In [ ]:
dusers = {x+1:y for x,y in enumerate(sorted(set(users)))}

We also load in a dictionary which maps from the artist integers to artist names. 

In [ ]:
import pickle
file = open("data/dartists.pkl","rb")
dartists = pickle.load(file)

We want to convert the integers representing artist names back into artist names, using the dictionary. 

We group the data set by user. From there we can see which artists a particular user has listened to:

In [ ]:
import numpy as np

def user_data(user, grouped_data, dusers):
    return grouped_data.get_group(dusers[user]) 

def top_k_listens(listening_history, k=10):
    top_k = listening_history.sort_values(by="plays", ascending = False)["artist"].head(k).values
    return artist_names(top_k)

def artist_names(artist_ints, artist_dic = dartists):
    return [artist_dic[k] for k in artist_ints]

In [ ]:
df.sample(10)

In [ ]:
grouped_df = df.groupby(['user'])

For a particular user we can have a look at their listening history as well as their most listened to artists. 

In [ ]:
import numpy as np
u100 = user_data(100, grouped_df, dusers)
u100_samp = u100.sample(10)

In [ ]:
u100_samp

In [ ]:
top_k_listens(u100, 10)

For each user, we want to generate a minhash of their listening history. The minhash class which we used in the previous notebook has been put into its own module for ease. 

In [ ]:
from datasketching.minhash import LSHMinhash
from datasketching.minhash import murmurmaker

In [ ]:
def generate_minhash_sig(user_dat, rows, bands):
    mh = LSHMinhash(rows, bands)
    for row in user_dat:
        mh.add(row)
    return mh

So for each user, we want to compose a list of all the artists they listened to. From there we will generate minhashes for each user, then make predictions. 

In [ ]:
un_artists = grouped_df['artist']

In [ ]:
for t in df.head(10).iloc[:, :]:
    print(t)

In [ ]:
%%time
from collections import defaultdict
from datasketching.minhash import LSHMinhash

# generate minhashes by processing every row of the data frame

minhashes = defaultdict(lambda: LSHMinhash(32, 4))

for user, artist in zip(df["user"], df["artist"]):
    minhashes[user].add(artist)

In [ ]:
## this next cell takes about ten minutes to run with 128 nhash
## 80 minutes with 1024 hash functions. 

In [ ]:
from datasketching.minhash import generate_minhashes_for

In [ ]:
%%time

# generate minhashes by combining single-artist signatures
proto = LSHMinhash(32,4)
sigs2 = generate_minhashes_for(grouped_df, "user", "artist", proto)

In [ ]:
len(mh_sigs)

In [ ]:
len(sigs2)
sigs2[0][1]

In [ ]:
minhashes[2].similarity(sigs2[1][1])

Once we have minhash signatures for all of the users we can compare them. But this isnt a quick process - suppose we want to find users who are similar to user 2. 

In [ ]:
sim=[]
for mh in range(1, len(mh_sigs)):
    sim.append(mh_sigs[dusers[mh]].similarity(mh_sigs[dusers[100]]))

Let's take a look at the users who are most similar to user 2:

In [ ]:
similar = set(sorted(sim, reverse = True)[1:10])
similar_users = ([i for i, e in enumerate(sim) if e in similar])
for j in similar_users:
    print(dusers[j])

These are the most similar users. Let's go ahead and look at the top artists listened to by all these users. 

Going to look at the unique artists listened to by each of these, remove uniques listened to by user 2, and then return the most listened across the other users. 

Look at the ?top 10 artists most listened to by these users that our user didnt listen to. 

In [ ]:
unheard = []
for u in similar_users:
    u_dat = user_data(u, grouped_df, dusers)
    unheard = unheard + list(top_k_listens(u_dat, 2))

In [ ]:
unheard

So that's just a quick example of how we can use minhash to identify songs we should recommend to a particular user. This method works fine on a small number of users, but falls into dificulty when the number of users grows, and the number of users for which we want to make recommendations for grows. 

## Locality-Sensitive Minhash

One big disadvantage of using Minhash signatures to identify similar users is the number of pairwise comparisons which must be made to determine similarity. 

Locality-sensitive Minhash is a technique we can use to identify candidate pairs of similar users for a much smaller computational cost. The method works by hashing subsets minhash signatures. If 2 users have identical signatures in ANY of the subsets these users are considered a candidate pair. And from there you can go and compute their approximate Jaccard index, or similarity, using the full minhash signatures, to determine just how similar they are, and decide if you want to make recommendations. 

The way in locality sensitive minhash works is by splitting the minhash signatures into bands. The bands are then hashed to buckets. 

if, in any band, two users map to the same bucket, they would be considered a candidate pair. At that point you’d go back and look at their minhash signatures, and compare those to determine how similar the users are. 


And thus we only have to compute the similarity of the minhash signatures for a subset of the whole population. 

In [ ]:
from datasketching.minhash import LSHMinhash
import random

In [ ]:
def lsmh(mh_sig, bands):
    ### assumes that the lenth of the mhsig is divisible by bands. 
    ### make more robust
    rows = int(len(mh_sig.buckets)/bands)
    return [mh_sig.hashes[0]([b for b in band]) for band in mh_sig.buckets.copy().reshape((rows, bands))]
    

In [ ]:
from collections import defaultdict

bands = [defaultdict(lambda: list()) for i in range(16)]

for ind, mh_sg in enumerate(mh_sigs):
    for idx, key in enumerate(lsmh(mh_sg, bands=16)):
        bands[idx][key % (1 << 14)].append(ind)

We've made a dictionary of values for each band, where the keys correspond to buckets, and the values are indexes of minhash signature which mapped to that bucket. 

If two minhash signatures hash to the same key in ANY band we consider them to be 'candidate pairs'. 

This means that the corresponding users _may_ be similar. We can check if they are similar by comparing the set of artists they have each listend to. From there we can use this information to make recomendations, or move on to consider other candidate pairs. 

In [ ]:
bands